# Oblivious Transfer (OT) and OT Extensions

This page implements several base oblivious transfer (OT) schemes and OT extensions which are needed when performing a large number of oblivious transfers.

## Semi-Honest Base OT Syntax and Semantics

Semi-Honest Alice ($\mathbb{A}$) has two single-bit messages $m_0$ and $m_1$. Semi-Honest Bob ($\mathbb{B}$) has a single bit choice bit $b$. Alice wants to send one -- and only one -- of her two messages to Bob. Bob, depending upon his choice bit $b$, wants to access $m_b$ but does not want Alice to know which message he picked. This interaction is summarized in the following diagram:

<img id="basic-ot" src="./Diagrams/BasicOT.svg" style="margin-left:auto; margin-right:auto"/>


The correctness requirements for an OT protocol are obvious: If Bob's choice bit is $b$, then $\forall m_0, m_1 \in \{0,1\}$, with high probability Bob should receive $m_b$. 

**Notation**


> * For a function $f$, inputs from different parties are separated by a semicolon '$;$'. In particular, oblivious transfer as a function is represented by following notation $$\textsf{OT}(m_0,m_1;\; b) \to m_b$$ where input $b$ is separated from $m_0$ and $m_1$ by a semi-colon to represent that it came from a different party. This notation can be extended to $k$ parties. 
> * Alice's view (transcript) while interacting with Bob is denoted as $$\langle\mathbb{A}, \mathbb{B}\rangle(m_0, m_1, r_{\mathbb{A}};\; b)$$ where $r_{\mathbb{A}}$ is Alice's private randomness during protocol execution.
> * Similarly Bob's view while interacting with Alice is denoted as $$\langle\mathbb{B}, \mathbb{A}\rangle(m_0, m_1;\; b, r_{\mathbb{B}})$$ where $r_{\mathbb{B}}$ is Bob's private randomness during protocol execution.

The privacy requirements for Alice and Bob are summarized below:

##### Privacy Requirements 

1. **Alice's Privacy**: Let Bob's choice bit be $b$ and let $\mathcal{D}$ be a polynomial time distinguisher that a corrupt Bob is trying to use to glean extra information about $m_{1-b}$ from its message transcript $$\tau := \langle \mathbb{B}, \mathbb{A}\rangle(m_0, m_1;\; b, r_\mathbb{B})$$ An OT scheme preserves Alice's privacy if, given $\tau$, the probability that $\mathcal{D}$ can distinguish $m_{1-b}$ from $1$ (or $0$) with probability significantly greater than $\frac{1}{2}$ is negligible, i.e.,  $$\forall \mathcal{D} \in \text{p.p.t} \wedge \forall \tau:\;\mathbf{Pr}\left[\mathcal{D}(b, m_b, \tau) = 1\right] < \frac{1}{2} + \textsf{negl}$$ where the probability is taken over the random choices made by $\mathcal{D}$.
2. **Bob's Privacy**: Let $\mathcal{D}'$ be a polynomial time distinguisher that a corrupt Alice is trying to use to find information about $b$ from its message transcript $$\tau' := \langle \mathbb{A}, \mathbb{B}\rangle(m_1, m_2, r_\mathbb{A};\; b)$$ An OT scheme preserves Bob's privacy if, given $\tau'$, the probability that $\mathcal{D}'$ can guess the value of $b$ with probability significantly greater than $\frac{1}{2}$ is negligible, i.e.,  $$\forall \mathcal{D}' \in \text{p.p.t}\wedge \forall \tau':\;\mathbf{Pr}\left[\mathcal{D}'(m_1, m_0, \tau') = b\right] < \frac{1}{2} + \textsf{negl}$$ where the probability is taken over the random choices made by $\mathcal{D}'$.

The following code abstracts this interface:

In [5]:
class ObliviousTransfer:
    """A basic interface for Oblivious transfer"""
    def __init__(self) -> None:
        """Compute the keys and other setup parameters"""
        pass

    def sender_setup_data(self, choice_bit):
        """Returns the public-key etc., that the sender needs to submit its
        messages"""
        raise NotImplemented

    def receiver_setup_data(self):
        """Returns the data that the Receiver needs to submit its
        messages"""
        raise NotImplemented

    def submit_messages(self, setup_data, msg0, msg1):
        """Takes message `msg0` and `msg1` and returns the transcript that can
        be used by Bob to recover one of `msg0` and `msg1`"""
        raise NotImplemented

    def choose_message(self, setup_data, sender_transcript, choice):
        """Takes the sender transcript that was generated during
        `submit_messages` and uses the choice bit to return receiver's
        transcript """
        raise NotImplemented

##### Theorem
> An oblivious transfer scheme where both the sender and the receiver are computationally unbounded is impossible.

We will prove this theorem in two stages: First we will show that given _any
oblivious transfer scheme_ $\textsf{OT}(m_0, m_1;\; b) \to m_b$, one can
construct a _two party MPC protocol_ $\textsf{2pc}_{\wedge}(a;\;b)$ that
securely computes $a\wedge b$ using $\textsf{OT}$ as a black box. Second, we
will prove that there _does not exist_ any information-theoretically secure MPC
protocol for computing two party $\textsf{and}$ gate. Combining these two
results, we can conclude that it's impossible to have information-theoretically
secure oblivious transfer protocol. 

> **Proof** [$\textsf{OT} \implies 2\textsf{pc}_\wedge$]: Suppose $\mathcal{P}_0$ has bit $b_0$ and $\mathcal{P}_1$ have bit $b_1$ and $\mathcal{P}_0$ and $\mathcal{P}_0$ want to use $\textsf{OT}(m_0, m_1;\; b)$ to _securely_ compute $b_0 \wedge b_1$. Here's how the two parties can proceed:
> 1. $\mathcal{P}_0$ acts as the $\textsf{OT}$ sender and $\mathcal{P}_1$ acts as the $\textsf{OT}$ receiver.
> 2. As an OT sender, $\mathcal{P}_0$ feeds $m_0 \leftarrow 0$ and $m_1 \leftarrow b_0$ to the oblivious transfer black box.
> 3. As an OT receiver, $\mathcal{P}_1$ feeds $b \leftarrow b_1$ into the OT black box.
>
> Claim: The output of above OT setup securely computes $b_0\wedge b_1$. Correctness holds because $\textsf{OT}(0, b_0;\; b_1) = \left [0\wedge (\neg b_1) \right ] \oplus \left [b_0\wedge b_1 \right ] = b_0\wedge b_1$. Since OT preserves sender and receiver's privacy, the privacy of user's inputs $b_0$ and $b_1$ are also preserved. Furthermore, depending on whether OT is maliciously secure or semi-honestly secure, secure function evaluation (SFE) of $\wedge$-gate achieves the same security goals.

Given this result, we now prove that its impossible to have an information theoretically secure two party $\wedge$ gate computation. 

#### Semi-honest Basic OT from RSA Hardcore predicate

1. **Setup**: Based on security parameter $\lambda$ (say $\lambda = 2048$), receiver generates two integers $S,$ and $T$ of $\lambda$-bit length as follows:
    * Sample a non-zero integer $T$ uniformly at random, i.e.,  $$T \xleftarrow{\$} \{1, \cdots, 2^\lambda  - 1\}$$ 
    * Sample two random $\frac{\lambda}{2}$-bit random prime numbers $p$ and $q$ and compute $S$ as follows: $$\begin{aligned} p,q &\xleftarrow{\$} \textsf{PRIMES}\{0, \cdots, 2^{\frac{\lambda}{2}}\} \\ S &= p\cdot q \end{aligned} $$
    
    * $\mathbb{B} \longrightarrow \mathbb{A}$: As part of the sender's setup, the receiver $\mathbb{B}$ sends the following setup data $(\Omega_0,\Omega1)$ to $\mathbb{A}$ depending upon its choice bit $b$: $$(\Omega_0, \Omega_1) = \begin{cases} (S,T) &\text{if } b = 0 \\ (T,S) &\text{if } b = 1 \end{cases}$$ Assuming $T$ is a large non-smooth composite numbers, a computationally bounded $\mathbb{A}$ cannot distinguish if $\mathbb{B}$ knows the factorization of $S$ or $T$. (Note, however, that a computationally unbounded $\mathbb{A}$ will know w.h.p. which is which, since a randomly sampled $T$ has a very different distribution from $S$ which is the product of exactly two $\frac{\lambda}{2}$-bit primes.) 